<a href="https://colab.research.google.com/github/Peace3B/Math_Tutor_LLM_Summative_Assignment/blob/main/Math_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Math Education Assistant: Fine-tuning LLM with LoRA**

Project Overview This notebook demonstrates fine-tuning a Large Language Model for math education using;

Dataset: UltraData-Math from Hugging Face

Base Model: TinyLlama-1.1B
Method: LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning
Domain: Mathematics Education

*Table of Contents*
  1. Setup and Installation
  2. Data Loading and Exploration
  3. Data Preprocessing
  4. Model Selection and LoRA Configuration
  5. Training with Hyperparameter Experiments
  6. Evaluation Metrics
  7. Model Comparison (Base vs Fine-tuned)
  8. Deployment with Gradio

**1. Setup and Installation**

Install required libraries for fine-tuning with LoRA and deployment.

In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes gradio evaluate rouge-score nltk torch pandas tqdm\n",
!pip install -q sentencepiece protobuf sacrebleu"

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [2]:
!pip install evaluate

In [3]:
# Import libraries
import os
import warnings
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import evaluate
import gradio as gr

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

# Suppress warnings
warnings.filterwarnings("ignore")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(
        f"GPU Memory: "
        f"{torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB"
    )

Using device: cuda
GPU: Tesla T4
GPU Memory: 15.64 GB


**2. Data Loading and Exploration**

Load the UltraData-Math dataset and explore its structure.

In [4]:
# Load library
from datasets import load_dataset

print("Loading UltraData-Math dataset in streaming mode...")

# Load training split with a specified config name in streaming mode
dataset = load_dataset("openbmb/UltraData-Math", name="UltraData-Math-L3-Conversation-Synthetic", split="train", streaming=True)

print(f"\nDataset features:")
print(dataset.features)

print("\nFirst 5 examples (from streaming data):")
# Iterate and print first few examples from the streaming dataset
for i, example in enumerate(dataset):
    print(example)
    if i >= 4: # Print 5 examples
        break


Loading UltraData-Math dataset in streaming mode...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]


Dataset features:
{'uid': Value('string'), 'content': Value('string')}

First 5 examples (from streaming data):
{'uid': '6f55f96b-a0eb-4b93-9af2-68367ae2e744', 'content': "Student: I'm having trouble understanding the problem statement for the integral \\(\\int_{0}^{2\\pi}(x-\\pi)^2 (\\sin x) dx\\). Could you explain it to me step-by-step?\n\nTeacher: Of course! Let's break it down. The problem is to evaluate the integral \\(\\int_{0}^{2\\pi}(x-\\pi)^2 (\\sin x) dx\\).\n\nStudent: Okay, I see the integral. But why do we need to use substitution here?\n\nTeacher: Good question. Using substitution can simplify the integral. We let \\( t = x - \\pi \\). This substitution helps us center the integral around zero, which can make it easier to apply symmetry properties.\n\nStudent: I see. So, what happens to the limits of integration when we make this substitution?\n\nTeacher: When \\( x = 0 \\), \\( t = -\\pi \\). When \\( x = 2\\pi \\), \\( t = \\pi \\). So, the new limits of integration a

**3. Data Preprocessing**

Convert the dataset into instruction-response format suitable for training.

In [5]:
# Define formatting function
def format_instruction(example):
    """
    Format dataset into instruction-response text, specifically for conversational math data.
    It extracts the first student/user turn as the instruction and the rest of the dialogue as the response.
    """

    instruction = None
    response = None

    if "content" in example: # Specific handling for UltraData-Math's 'content' field
        dialogue = example["content"]

        # Find the first occurrence of "Student:" or "User:"
        student_idx = dialogue.find("Student:")
        user_idx = dialogue.find("User:")

        first_speaker_idx = -1
        speaker_tag_len = 0

        if student_idx != -1 and (user_idx == -1 or student_idx < user_idx):
            first_speaker_idx = student_idx
            speaker_tag_len = len("Student:")
        elif user_idx != -1 and (student_idx == -1 or user_idx < student_idx):
            first_speaker_idx = user_idx
            speaker_tag_len = len("User:")

        if first_speaker_idx != -1:
            # Find the start of the first response from a teacher/Qwen after the initial instruction
            teacher_idx = dialogue.find("Teacher:", first_speaker_idx + speaker_tag_len)
            qwen_idx = dialogue.find("Qwen:", first_speaker_idx + speaker_tag_len)

            response_start_idx = -1
            if teacher_idx != -1 and (qwen_idx == -1 or teacher_idx < qwen_idx):
                response_start_idx = teacher_idx
            elif qwen_idx != -1 and (teacher_idx == -1 or qwen_idx < teacher_idx):
                response_start_idx = qwen_idx

            if response_start_idx != -1:
                instruction = dialogue[:response_start_idx].strip()
                response = dialogue[response_start_idx:].strip()
            else:
                # If no clear teacher/Qwen response found, treat the whole dialogue as instruction
                instruction = dialogue.strip()
                response = ""
        else:
            # Fallback if no specific speaker turn is found (e.g., malformed content)
            instruction = "Analyze the following math dialogue:"
            response = dialogue.strip()

    # Fallback for other potential datasets with standard instruction/output fields
    elif "instruction" in example and "output" in example:
        instruction = example["instruction"]
        response = example["output"]

    elif "question" in example and "answer" in example:
        instruction = example["question"]
        response = example["answer"]

    elif "input" in example and "output" in example:
        instruction = example["input"]
        response = example["output"]

    # If after all attempts, instruction or response is still None, return an invalid example
    if instruction is None or response is None:
        return {"text": None}

    formatted_text = (
        "### Instruction:\n"
        f"{instruction}\n\n"
        "### Response:\n"
        f"{response}"
    )
    return {"text": formatted_text}

In [6]:
print("Formatting dataset...")

formatted_dataset = dataset.map(
    format_instruction,
    remove_columns=dataset.column_names
)

# Remove failed examples
formatted_dataset = formatted_dataset.filter(
    lambda x: x["text"] is not None
)

# For streaming datasets, we cannot directly get the length or access by index.
# We can iterate to get a few examples.
print("\nExample formatted text (first example from stream):\n")
for i, example in enumerate(formatted_dataset):
    print(example["text"][:500])
    if i == 0: # Print only the first example
        break

Formatting dataset...

Example formatted text (first example from stream):

### Instruction:
Student: I'm having trouble understanding the problem statement for the integral \(\int_{0}^{2\pi}(x-\pi)^2 (\sin x) dx\). Could you explain it to me step-by-step?

### Response:
Teacher: Of course! Let's break it down. The problem is to evaluate the integral \(\int_{0}^{2\pi}(x-\pi)^2 (\sin x) dx\).

Student: Okay, I see the integral. But why do we need to use substitution here?

Teacher: Good question. Using substitution can simplify the integral. We let \( t = x - \pi \). Thi


In [7]:
# Select subset for efficient training
DATASET_SIZE = 3000  # Adjust depending on GPU memory

# For streaming datasets, we cannot directly use len(), shuffle() or train_test_split()
# if we want to limit the total number of examples and then split.
# We will take DATASET_SIZE examples from the stream and then convert it to a regular Dataset
# to enable shuffling and splitting.

print(f"Collecting {DATASET_SIZE} examples from the streaming dataset...")
# Using `take(DATASET_SIZE)` to get the first N examples from the stream.
# Then convert to a regular Dataset.
# Note: `shuffle()` on an IterableDataset shuffles the *order of chunks*, not individual examples.
# To get a truly random subset and then split, it's better to materialize a limited number.
limited_dataset_list = []
for i, example in enumerate(formatted_dataset):
    if i >= DATASET_SIZE:
        break
    limited_dataset_list.append(example)

# Convert the list of examples to a regular Dataset
from datasets import Dataset
materialized_dataset = Dataset.from_list(limited_dataset_list)
print(f"Materialized dataset size: {len(materialized_dataset)} examples")

# Now we can shuffle and split the materialized dataset
shuffled_dataset = materialized_dataset.shuffle(seed=42)

# Split into train and validation
split_dataset = shuffled_dataset.train_test_split(
    test_size=0.1,
    seed=42
)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"\nTraining examples: {len(train_dataset)}")
print(f"Validation examples: {len(eval_dataset)}")

Materialized dataset size: 3000 examples

Training examples: 2700
Validation examples: 300


**4. Model Selection and LoRA Configuration**

Load TinyLlama model and configure LoRA for parameter-efficient fine-tuning.

In [19]:
!  pip install -U bitsandbytes>=0.46.1

In [20]:
! pip install -U bitsandbytes

In [8]:
!pip install -U bitsandbytes accelerate

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"Loading model: {MODEL_NAME}")

# ---- 4-bit quantization config (modern way) ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# ---- Load tokenizer ----
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

# Fix padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# ---- Load model ----
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)

print("Model loaded successfully!")
print(f"Total parameters: {model.num_parameters() / 1e6:.2f}M")

Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!
Total parameters: 1100.05M


In [10]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

print("\nLoRA configuration applied successfully!")

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079

LoRA configuration applied successfully!


In [11]:
MAX_LENGTH = 300

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH
    )

    # Important for causal LM
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized


print("Tokenizing datasets...")

tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print("Tokenization complete!")

Tokenizing datasets...


Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenization complete!


In [12]:
import pandas as pd

experiment_results = []

def create_experiment_table():
    columns = [
        "Experiment",
        "Learning Rate",
        "Batch Size",
        "Gradient Accumulation",
        "Epochs",
        "LoRA Rank",
        "Final Train Loss",
        "Final Eval Loss",
        "Training Time (min)",
        "GPU Memory (GB)",
        "Notes"
    ]
    return pd.DataFrame(columns=columns)

experiment_table = create_experiment_table()
print("Experiment tracking table created!")

Experiment tracking table created!


**5. Training with Hyperparameter Experiments**

We'll conduct multiple training experiments with different hyperparameters and document results.

In [16]:
import time
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

def run_experiment(
    experiment_name,
    learning_rate,
    batch_size,
    gradient_accumulation_steps,
    num_epochs,
    lora_rank,
    notes=""
):

    print("\n" + "="*60)
    print(f"Starting {experiment_name}")
    print("="*60)

    torch.cuda.empty_cache()

    # ---- 4-bit config ----
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    # ---- Reload fresh model ----
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    model = prepare_model_for_kbit_training(model)

    # ---- Apply LoRA ----
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    model = get_peft_model(model, lora_config)

    # ---- Training arguments ----
    training_args = TrainingArguments(
        output_dir=f"./results/{experiment_name}",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        fp16=True,
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=400,
        save_total_limit=2,
        load_best_model_at_end=True,
        report_to="none",
        warmup_ratio=0.05,
        lr_scheduler_type="cosine"
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        data_collator=data_collator
    )

    # ---- Train ----
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    start_time = time.time()
    train_result = trainer.train()
    training_time = (time.time() - start_time) / 60

    final_train_loss = train_result.training_loss
    eval_results = trainer.evaluate()
    final_eval_loss = eval_results["eval_loss"]

    if torch.cuda.is_available():
        gpu_memory = torch.cuda.max_memory_allocated() / 1e9
    else:
        gpu_memory = 0

    # ---- Store results ----
    result_row = {
        "Experiment": experiment_name,
        "Learning Rate": learning_rate,
        "Batch Size": batch_size,
        "Gradient Accumulation": gradient_accumulation_steps,
        "Epochs": num_epochs,
        "LoRA Rank": lora_rank,
        "Final Train Loss": round(final_train_loss, 4),
        "Final Eval Loss": round(final_eval_loss, 4),
        "Training Time (min)": round(training_time, 2),
        "GPU Memory (GB)": round(gpu_memory, 2),
        "Notes": notes
    }

    global experiment_table
    experiment_table = pd.concat(
        [experiment_table, pd.DataFrame([result_row])],
        ignore_index=True
    )

    trainer.save_model(f"./models/{experiment_name}")

    print(f"\n{experiment_name} completed!")
    print(f"Training Loss: {final_train_loss:.4f}")
    print(f"Validation Loss: {final_eval_loss:.4f}")
    print(f"Training Time: {training_time:.2f} min")

    return trainer

In [17]:
trainer_exp1 = run_experiment(
    experiment_name="Exp1_Baseline",
    learning_rate=2e-4,
    batch_size=4,
    gradient_accumulation_steps=4,
    num_epochs=3,
    lora_rank=16,
    notes="Baseline configuration"
)


Starting Exp1_Baseline


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Step,Training Loss,Validation Loss
200,1.181927,1.193019
400,1.112981,1.177031



Exp1_Baseline completed!
Training Loss: 1.1664
Validation Loss: 1.1770
Training Time: 34.79 min


In [18]:
trainer_exp2 = run_experiment(
    experiment_name="Exp2_LowerLR",
    learning_rate=1e-4,
    batch_size=4,
    gradient_accumulation_steps=4,
    num_epochs=3,
    lora_rank=16,
    notes="Lower learning rate"
)


Starting Exp2_LowerLR


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Step,Training Loss,Validation Loss
200,1.205436,1.208861
400,1.157011,1.192853



Exp2_LowerLR completed!
Training Loss: 1.1994
Validation Loss: 1.1929
Training Time: 34.71 min


In [19]:
trainer_exp3 = run_experiment(
    experiment_name="Exp3_HigherRank",
    learning_rate=2e-4,
    batch_size=4,
    gradient_accumulation_steps=4,
    num_epochs=3,
    lora_rank=32,
    notes="Higher LoRA rank"
)


Starting Exp3_HigherRank


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Step,Training Loss,Validation Loss
200,1.181142,1.192468
400,1.110612,1.176073



Exp3_HigherRank completed!
Training Loss: 1.1651
Validation Loss: 1.1761
Training Time: 35.26 min


In [20]:
# Display experiment results table
print("\n" + "="*100)
print("EXPERIMENT RESULTS SUMMARY")
print("="*100)
print(experiment_table.to_string(index=False))

# Save to CSV
experiment_table.to_csv('experiment_results.csv', index=False)
print("\nResults saved to 'experiment_results.csv'")


EXPERIMENT RESULTS SUMMARY
     Experiment  Learning Rate Batch Size Gradient Accumulation Epochs LoRA Rank  Final Train Loss  Final Eval Loss  Training Time (min)  GPU Memory (GB)                  Notes
  Exp1_Baseline         0.0002          4                     4      3        16            1.1664           1.1770                34.79             4.07 Baseline configuration
   Exp2_LowerLR         0.0001          4                     4      3        16            1.1994           1.1929                34.71             5.17    Lower learning rate
Exp3_HigherRank         0.0002          4                     4      3        32            1.1651           1.1761                35.26             6.34       Higher LoRA rank

Results saved to 'experiment_results.csv'


**6. Evaluation Metrics**

Evaluate the best model using ROUGE, BLEU, and perplexity metrics.

In [22]:
! pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=62ef01a6be18618149c689dbcc2cf2ac0539ffa6717f0bb38186d073ef6df233
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [24]:
! pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.9 MB/s eta 0:00:00


In [25]:
# Load evaluation metrics
rouge_metric = evaluate.load('rouge')
bleu_metric = evaluate.load('sacrebleu')

print("Evaluation metrics loaded!")

Evaluation metrics loaded!


In [27]:
# Select best model (lowest eval loss)
best_experiment = experiment_table.loc[experiment_table['Final Eval Loss'].astype(float).idxmin()]
best_model_path = f"./models/{best_experiment['Experiment']}"

print(f"Best model: {best_experiment['Experiment']}")
print(f"Eval Loss: {best_experiment['Final Eval Loss']}")

# Load the best model for evaluation
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config, # Use the already defined bnb_config
    device_map="auto",
    trust_remote_code=True
)

fine_tuned_model = PeftModel.from_pretrained(base_model, best_model_path)
fine_tuned_model.eval()

print("Best model loaded for evaluation!")

Best model: Exp3_HigherRank
Eval Loss: 1.1761


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Best model loaded for evaluation!


In [29]:
# Generate predictions for evaluation
def generate_response(model, tokenizer, instruction, max_length=200):
    """
    Generate response from model given an instruction.
    """
    prompt = f"""### Instruction:
{instruction}

### Response:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the response part
    response = response.split("### Response:")[-1].strip()

    return response

# Test generation
# Extract instruction from the 'text' field of the eval_dataset example
example_text = eval_dataset[0]['text']
# The instruction is everything between '### Instruction:' and '### Response:'
parts = example_text.split("### Response:", 1)
instruction_part = parts[0]
test_instruction = instruction_part.split("### Instruction:\n", 1)[1].strip()

test_response = generate_response(fine_tuned_model, tokenizer, test_instruction)

print("Test generation:")
print(f"Instruction: {test_instruction}")
print(f"Generated Response: {test_response[:200]}...")

Test generation:
Instruction: Analyze the following math dialogue:
Generated Response: Professor A: Good afternoon, Professor B. I was reviewing some mathematical problems and came across a difficult one. The problem involves two people, John and Tom, who are both interested in solving ...


In [31]:
# Calculate metrics on evaluation set (sample for efficiency)
EVAL_SAMPLES = 50  # Adjust based on computational resources

predictions = []
references = []

print(f"Generating predictions for {EVAL_SAMPLES} samples...")
for i in tqdm(range(min(EVAL_SAMPLES, len(eval_dataset)))): # Note: len() works on materialized dataset
    example = eval_dataset[i]

    # Extract instruction and reference from the 'text' field
    example_text = example['text']
    parts = example_text.split("### Response:", 1)

    instruction_part = parts[0]
    # Ensure instruction_part is not empty before splitting further
    if "### Instruction:" in instruction_part:
        instruction = instruction_part.split("### Instruction:\n", 1)[1].strip()
    else:
        # Fallback if instruction format is unexpected, or skip example
        instruction = ""
        # Optionally, you might want to log a warning or skip this example

    # The reference is the part after "### Response:"
    reference = parts[1].strip() if len(parts) > 1 else ""

    # Ensure both instruction and reference are not empty
    if not instruction or not reference:
        tqdm.write(f"Skipping example {i} due to empty instruction or reference.")
        continue

    prediction = generate_response(fine_tuned_model, tokenizer, instruction)

    predictions.append(prediction)
    references.append(reference)

print("\nPredictions generated!")

Generating predictions for 50 samples...


 40%|████      | 20/50 [06:57<09:30, 19.01s/it]

Skipping example 20 due to empty instruction or reference.


100%|██████████| 50/50 [16:02<00:00, 19.25s/it]


Predictions generated!


In [32]:
# Calculate ROUGE scores
rouge_results = rouge_metric.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)

print("\n" + "="*50)
print("ROUGE SCORES")
print("="*50)
for key, value in rouge_results.items():
    print(f"{key}: {value:.4f}")


ROUGE SCORES
rouge1: 0.1963
rouge2: 0.0399
rougeL: 0.1182
rougeLsum: 0.1842


In [33]:
# Calculate BLEU score
# sacrebleu expects references as list of lists
bleu_results = bleu_metric.compute(
    predictions=predictions,
    references=[[ref] for ref in references]
)

print("\n" + "="*50)
print("BLEU SCORE")
print("="*50)
print(f"BLEU: {bleu_results['score']:.4f}")


BLEU SCORE
BLEU: 0.1746


In [34]:
# Calculate Perplexity
import math

def calculate_perplexity(model, tokenized_dataset, num_samples=100):
    """
    Calculate perplexity on a dataset.
    """
    model.eval()
    total_loss = 0
    total_tokens = 0

    with torch.no_grad():
        for i in tqdm(range(min(num_samples, len(tokenized_dataset))), desc="Calculating perplexity"):
            example = tokenized_dataset[i]
            input_ids = torch.tensor([example['input_ids']]).to(model.device)
            labels = torch.tensor([example['labels']]).to(model.device)

            outputs = model(input_ids, labels=labels)
            loss = outputs.loss

            total_loss += loss.item() * len(example['input_ids'])
            total_tokens += len(example['input_ids'])

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)

    return perplexity

perplexity = calculate_perplexity(fine_tuned_model, tokenized_eval)

print("\n" + "="*50)
print("PERPLEXITY")
print("="*50)
print(f"Perplexity: {perplexity:.4f}")

Calculating perplexity: 100%|██████████| 100/100 [00:14<00:00,  6.97it/s]


PERPLEXITY
Perplexity: 3.5487


In [35]:
# Create evaluation summary
evaluation_summary = pd.DataFrame([
    {'Metric': 'ROUGE-1', 'Score': f"{rouge_results['rouge1']:.4f}"},
    {'Metric': 'ROUGE-2', 'Score': f"{rouge_results['rouge2']:.4f}"},
    {'Metric': 'ROUGE-L', 'Score': f"{rouge_results['rougeL']:.4f}"},
    {'Metric': 'BLEU', 'Score': f"{bleu_results['score']:.4f}"},
    {'Metric': 'Perplexity', 'Score': f"{perplexity:.4f}"}
])

print("\n" + "="*50)
print("EVALUATION SUMMARY")
print("="*50)
print(evaluation_summary.to_string(index=False))

evaluation_summary.to_csv('evaluation_metrics.csv', index=False)
print("\nEvaluation metrics saved to 'evaluation_metrics.csv'")


EVALUATION SUMMARY
    Metric  Score
   ROUGE-1 0.1963
   ROUGE-2 0.0399
   ROUGE-L 0.1182
      BLEU 0.1746
Perplexity 3.5487

Evaluation metrics saved to 'evaluation_metrics.csv'


**7. Model Comparison: Base vs Fine-tuned**

Compare responses from the base model and fine-tuned model.

In [37]:
# Load base model for comparison
base_model_for_comparison = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config, # Use the already defined bnb_config
    device_map="auto",
    trust_remote_code=True
)
base_model_for_comparison.eval()

print("Base model loaded for comparison!")

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Base model loaded for comparison!


In [38]:
# Test on sample math questions
test_questions = [
    "What is the Pythagorean theorem?",
    "How do you solve a quadratic equation?",
    "Explain the concept of derivatives in calculus.",
    "What is the difference between mean and median?",
    "Solve for x: 2x + 5 = 15"
]

print("\n" + "="*100)
print("BASE MODEL vs FINE-TUNED MODEL COMPARISON")
print("="*100)

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*100}")
    print(f"Question {i}: {question}")
    print(f"{'='*100}")

    # Base model response
    base_response = generate_response(base_model_for_comparison, tokenizer, question, max_length=150)
    print(f"\n[BASE MODEL]\n{base_response}")

    # Fine-tuned model response
    finetuned_response = generate_response(fine_tuned_model, tokenizer, question, max_length=150)
    print(f"\n[FINE-TUNED MODEL]\n{finetuned_response}")
    print()


BASE MODEL vs FINE-TUNED MODEL COMPARISON

Question 1: What is the Pythagorean theorem?

[BASE MODEL]
The Pythagorean theorem is a mathematical theorem that relates the sides of a right-angled triangle to each other, and the length of the perpendicular between them. The theorem states that the square of the hypotenuse (side opposite the right angle) is equal to the sum of the squares of the other two sides, which is equal to the square of the longer side multiplied by a constant known as the hypotenuse ratio. This ratio is called the hypotenuse ratio or the length of the perpendicular.

[FINE-TUNED MODEL]
The Pythagorean theorem is a theorem in geometry that states that in a right-angled triangle, the square of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the other two sides. This theorem is named after the ancient Greek mathematician Pythagoras.

The theorem is expressed as follows:
$$
\text{Hypotenuse}^2 = \text{Square of hypotenuse} + \te

In [39]:
# Qualitative analysis on out-of-domain queries
out_of_domain_questions = [
    "What's the weather like today?",
    "Tell me a joke.",
    "What is the capital of France?"
]

print("\n" + "="*100)
print("OUT-OF-DOMAIN QUERY HANDLING")
print("="*100)

for i, question in enumerate(out_of_domain_questions, 1):
    print(f"\n{'='*100}")
    print(f"Question {i}: {question}")
    print(f"{'='*100}")

    response = generate_response(fine_tuned_model, tokenizer, question, max_length=100)
    print(f"\n[RESPONSE]\n{response}")


OUT-OF-DOMAIN QUERY HANDLING

Question 1: What's the weather like today?

[RESPONSE]
The weather is going to be very nice, with temperatures ranging from 80°F to 85°F. The humidity is 56%, which means the air is very moist. The wind is light, with a speed of 10 mph. The sky is clear, and the clouds are white. The forecast is for a mostly sunny day.

I hope that helps. If you have any more questions, feel free to ask

Question 2: Tell me a joke.

[RESPONSE]
Alice: What do you call a pink elephant with a pink nose and a pink tongue?
Bob: An elephant with a pink nose and a pink tongue.
Alice: Oh, I see. So, what's the difference between a pink elephant and a pink panda?
Bob: A pink panda has a pink nose and a pink tongue, while a pink elephant has a

Question 3: What is the capital of France?

[RESPONSE]
The capital of France is Paris, which is the most populous city in France and the capital of the Île-de-France region.

### Notes:
- "Paris" is the capital of France, not just the city o

**8. Deployment with Gradio**

Create an interactive web interface for the Math Education Assistant.

In [40]:
# Prepare model for deployment
# Merge LoRA weights with base model for faster inference
final_model = fine_tuned_model.merge_and_unload()
final_model.eval()

print("Model prepared for deployment!")

Model prepared for deployment!


In [43]:
# Define chatbot function for Gradio
def math_tutor_chatbot(message, history):
    """
    Chatbot function for Gradio interface.

    Args:
        message: User's current message
        history: List of previous [user_msg, assistant_msg] pairs

    Returns:
        Generated response
    """
    # Generate response
    response = generate_response(final_model, tokenizer, message, max_length=300)
    return response

# Create Gradio interface
demo = gr.ChatInterface(
    fn=math_tutor_chatbot,
    title="Math Education Assistant",
    description="""Ask me any math question! I'm fine-tuned on mathematical problems and can help with:
    - Algebra and equations
    - Geometry and trigonometry
    - Calculus concepts
    - Statistics and probability
    - Problem-solving strategies

    **Note**: This is an AI assistant - always verify important calculations!""",
    examples=[
        "What is the quadratic formula?",
        "How do I calculate the area of a circle?",
        "Explain what a derivative is.",
        "Solve: 3x - 7 = 11",
        "What's the difference between permutations and combinations?"
    ],
    theme="soft"
    # Removed unsupported keyword arguments: retry_btn="Retry", undo_btn="Undo", clear_btn="Clear"
)

# Launch the interface
print("\nLaunching Math Education Assistant...")
demo.launch(share=True, debug=True)


Launching Math Education Assistant...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://57b54929f569825073.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 